In [1]:
!pip install catboost -q

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
DIR = "/content/drive/My Drive/jobathon-june2022/"

train = pd.read_csv(DIR + "train.csv")
test = pd.read_csv(DIR + "test.csv")

test["buy"] = -1

df = pd.concat([train,test]).reset_index(drop=True)
df

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy
0,1,2021-01-01,1,2,2.0,2020-09-24,0,0,0,0,0,0,1,1,0,0,0,0,0
1,2,2021-01-01,2,1,2.0,2020-09-19,1,0,1,0,0,0,1,0,0,0,0,0,0
2,3,2021-01-01,9,3,3.0,2021-08-11,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,2021-01-01,6,7,2.0,2017-10-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,2021-01-01,4,6,NaN,2020-06-08,0,0,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52340,52341,2022-03-30,16,12,NaN,NaN,0,0,0,0,0,1,1,0,0,0,0,0,-1
52341,52342,2022-03-30,12,8,NaN,2022-03-19,1,0,0,0,0,1,0,0,0,0,0,0,-1
52342,52343,2022-03-30,14,14,3.0,2021-08-15,0,0,1,0,0,0,1,0,0,0,0,0,-1
52343,52344,2022-03-30,14,10,NaN,NaN,0,0,0,0,0,0,1,0,0,0,1,0,-1


In [5]:
df.isnull().sum()

id                          0
created_at                  0
campaign_var_1              0
campaign_var_2              0
products_purchased      29047
signup_date             21762
user_activity_var_1         0
user_activity_var_2         0
user_activity_var_3         0
user_activity_var_4         0
user_activity_var_5         0
user_activity_var_6         0
user_activity_var_7         0
user_activity_var_8         0
user_activity_var_9         0
user_activity_var_10        0
user_activity_var_11        0
user_activity_var_12        0
buy                         0
dtype: int64

In [6]:
date_cols = ["created_at","signup_date"]

for col in df.columns.to_list():
    if col not in date_cols + ["id"]:
    print(col,"   ",df[col].nunique())

campaign_var_1     16
campaign_var_2     15
products_purchased     4
user_activity_var_1     4
user_activity_var_2     2
user_activity_var_3     2
user_activity_var_4     2
user_activity_var_5     3
user_activity_var_6     4
user_activity_var_7     3
user_activity_var_8     3
user_activity_var_9     3
user_activity_var_10     2
user_activity_var_11     5
user_activity_var_12     2
buy     3


In [7]:
for c in date_cols:
    df[c] = pd.to_datetime(df[c])

In [8]:
df["date_diff"] = (df["created_at"] - df["signup_date"]).dt.days
df

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy,date_diff
0,1,2021-01-01,1,2,2.0,2020-09-24,0,0,0,0,0,0,1,1,0,0,0,0,0,99.0
1,2,2021-01-01,2,1,2.0,2020-09-19,1,0,1,0,0,0,1,0,0,0,0,0,0,104.0
2,3,2021-01-01,9,3,3.0,2021-08-11,1,0,0,0,0,0,0,0,0,0,0,0,0,-222.0
3,4,2021-01-01,6,7,2.0,2017-10-04,0,0,0,0,0,0,0,0,0,0,0,0,0,1185.0
4,5,2021-01-01,4,6,NaN,2020-06-08,0,0,0,0,0,0,1,0,0,0,1,0,0,207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52340,52341,2022-03-30,16,12,NaN,NaT,0,0,0,0,0,1,1,0,0,0,0,0,-1,NaN
52341,52342,2022-03-30,12,8,NaN,2022-03-19,1,0,0,0,0,1,0,0,0,0,0,0,-1,11.0
52342,52343,2022-03-30,14,14,3.0,2021-08-15,0,0,1,0,0,0,1,0,0,0,0,0,-1,227.0
52343,52344,2022-03-30,14,10,NaN,NaT,0,0,0,0,0,0,1,0,0,0,1,0,-1,NaN


In [9]:
for c in date_cols:
    df[f"{c}_day"] = df[c].dt.day
    df[f"{c}_month"] = df[c].dt.month

In [10]:
df.columns

Index(['id', 'created_at', 'campaign_var_1', 'campaign_var_2',
       'products_purchased', 'signup_date', 'user_activity_var_1',
       'user_activity_var_2', 'user_activity_var_3', 'user_activity_var_4',
       'user_activity_var_5', 'user_activity_var_6', 'user_activity_var_7',
       'user_activity_var_8', 'user_activity_var_9', 'user_activity_var_10',
       'user_activity_var_11', 'user_activity_var_12', 'buy', 'date_diff',
       'created_at_day', 'created_at_month', 'signup_date_day',
       'signup_date_month'],
      dtype='object')

In [13]:
df["products_purchased"] = df["products_purchased"].astype(str)
df.dtypes

id                               int64
created_at              datetime64[ns]
campaign_var_1                   int64
campaign_var_2                   int64
products_purchased              object
signup_date             datetime64[ns]
user_activity_var_1              int64
user_activity_var_2              int64
user_activity_var_3              int64
user_activity_var_4              int64
user_activity_var_5              int64
user_activity_var_6              int64
user_activity_var_7              int64
user_activity_var_8              int64
user_activity_var_9              int64
user_activity_var_10             int64
user_activity_var_11             int64
user_activity_var_12             int64
buy                              int64
date_diff                      float64
created_at_day                   int64
created_at_month                 int64
signup_date_day                float64
signup_date_month              float64
cum-activity                     int64
dtype: object

In [12]:
df["cum-activity"] = df.apply(lambda x: [x[f"user_activity_var_{a}"] for a in range(1,13)],axis=1)
df["cum-activity"] = df["cum-activity"].apply(lambda x: np.sum(x))

In [14]:
cat_cols = ['products_purchased','user_activity_var_1','user_activity_var_2', 'user_activity_var_3', 'user_activity_var_4','user_activity_var_5', 'user_activity_var_6', 'user_activity_var_7',
       'user_activity_var_8', 'user_activity_var_9', 'user_activity_var_10','user_activity_var_11', 'user_activity_var_12','cum-activity']
target = ["buy"]
num_cols = [c for c in df.columns.to_list() if c not in date_cols + cat_cols + target + ["id"]]
num_cols

['campaign_var_1',
 'campaign_var_2',
 'date_diff',
 'created_at_day',
 'created_at_month',
 'signup_date_day',
 'signup_date_month']

In [15]:
from catboost import CatBoostClassifier
import copy

all_cols = num_cols + cat_cols
c = "products_purchased"
all_cols.remove(c)
categorical_columns = [col for col in cat_cols if col != c]

m = CatBoostClassifier(task_type="GPU",devices='0:1')

train_features = df.loc[~pd.isna(df[c]),all_cols].reset_index(drop=True)
train_prediction =  df.loc[~pd.isna(df[c]),c].values
m.fit(train_features,train_prediction,verbose=100,cat_features=categorical_columns)
test_features = df.loc[pd.isna(df[c]),:].reset_index(drop=True)
test_prediction = m.predict(test_features.loc[:,all_cols])
df.loc[pd.isna(df[c]),c] = test_prediction

Learning rate set to 0.145524
0:	learn: 1.4608277	total: 89ms	remaining: 1m 28s
100:	learn: 1.0990175	total: 4.36s	remaining: 38.8s
200:	learn: 1.0777273	total: 7.87s	remaining: 31.3s
300:	learn: 1.0583308	total: 9.04s	remaining: 21s
400:	learn: 1.0386377	total: 10.1s	remaining: 15.1s
500:	learn: 1.0203389	total: 11.2s	remaining: 11.2s
600:	learn: 1.0005119	total: 12.3s	remaining: 8.17s
700:	learn: 0.9828467	total: 13.4s	remaining: 5.73s
800:	learn: 0.9651249	total: 14.5s	remaining: 3.61s
900:	learn: 0.9476677	total: 15.7s	remaining: 1.72s
999:	learn: 0.9306806	total: 16.8s	remaining: 0us


In [16]:
df.isnull().sum()

id                          0
created_at                  0
campaign_var_1              0
campaign_var_2              0
products_purchased          0
signup_date             21762
user_activity_var_1         0
user_activity_var_2         0
user_activity_var_3         0
user_activity_var_4         0
user_activity_var_5         0
user_activity_var_6         0
user_activity_var_7         0
user_activity_var_8         0
user_activity_var_9         0
user_activity_var_10        0
user_activity_var_11        0
user_activity_var_12        0
buy                         0
date_diff               21762
created_at_day              0
created_at_month            0
signup_date_day         21762
signup_date_month       21762
cum-activity                0
dtype: int64

In [30]:
fea_cols = ["user_activity_var_1","user_activity_var_5","user_activity_var_7","user_activity_var_9","user_activity_var_12"]
addl_cat_cols = []
for id1 in range(len(fea_cols)):
    for id2 in range(id1,len(fea_cols),1):
    c1 = fea_cols[id1]
    c2 = fea_cols[id2]
    df[c1 + "_+_" + c2] = df.apply(lambda x: str(x[c1]) + "_+_" + str(x[c2]),axis=1)
    addl_cat_cols.append(c1 + "_+_" + c2)


In [31]:
cat_cols = cat_cols + addl_cat_cols

In [32]:
train = df.loc[df["buy"]!=-1,num_cols + cat_cols + target].reset_index(drop=True)
test = df.loc[df["buy"]==-1,num_cols + cat_cols + target].reset_index(drop=True)
train.shape,test.shape

((39161, 37), (13184, 37))

In [33]:
train["buy"].value_counts()[0]/train["buy"].value_counts()[1]

18.6001001001001

In [19]:
def get_optim_threshold(y_val,fold_preds):

    thresholds = [0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95]
    optim_threshold = 0
    f1_ = 0
    for th in thresholds:
    f1_new = f1_score(y_val,(fold_preds > th).astype(int))
    if f1_new > f1_:
        f1_ = f1_new
        optim_threshold = th

    print("Optimal threshold: ",optim_threshold)
    return optim_threshold


In [20]:
def lgb_f1_score(y_true,y_preds):
    y_preds = np.round(y_preds) # scikits f1 doesn't like probabilities
    #print(y_true)
    return 'f1', f1_score(y_true, y_preds), True

In [25]:
class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets,weights):
        #assert len(approxes) == len(targets)
        #if weights is not None:
         #   assert len(weights) == len(approxes)
        
        weights = np.where(targets==1,targets*8,targets+1)

        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = targets[index] - p
            der2 = -p * (1 - p)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [36]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
import lightgbm as lgb


skf = StratifiedKFold(n_splits=5)
test = test[num_cols + cat_cols]

oof_scores = []
preds = np.zeros(shape=(len(test),))

for fold,(trn_idx,val_idx) in enumerate(skf.split(train[num_cols + cat_cols],train[target])):
    print("In fold ",fold+1)
    X_train,X_val = train.loc[trn_idx,num_cols + cat_cols],train.loc[val_idx,num_cols + cat_cols]
    y_train,y_val = train.loc[trn_idx,target],train.loc[val_idx,target]
    #lgb_clf = lgb.LGBMClassifier(n_estimators=300,metric="None")
    #lgb_clf.fit(X_train,y_train,eval_set=[(X_val,y_val)],categorical_feature = cat_cols, verbose = 50,eval_metric=lgb_f1_score,callbacks=[lgb.early_stopping(stopping_rounds=50)])
    clf = CatBoostClassifier(eval_metric="F1",random_state=fold+1)
    clf.fit(X_train,y_train,eval_set=[(X_val,y_val)],cat_features = cat_cols,early_stopping_rounds=250,verbose=200)
    #fold_preds = clf.predict_proba(X_val)[:,1]
    #optimal_threshold = get_optim_threshold(y_val.values,fold_preds)
    #fold_score = f1_score(y_val,(clf.predict_proba(X_val)[:,1] > optimal_threshold).astype(int))
    fold_preds = clf.predict(X_val)
    fold_score = f1_score(y_val,fold_preds)
    print(f"Fold {fold+1} score: ",fold_score)
    oof_scores.append(fold_score)
    preds += clf.predict(test)

print("Overall OOF score is ",np.array(oof_scores).mean())
preds = preds / 5

In fold  1
Learning rate set to 0.0742
0:	learn: 0.5277402	test: 0.5438596	best: 0.5438596 (0)	total: 223ms	remaining: 3m 43s
200:	learn: 0.7463264	test: 0.6221010	best: 0.6731079 (55)	total: 23.9s	remaining: 1m 34s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.6731078905
bestIteration = 55

Shrink model to first 56 iterations.
Fold 1 score:  0.6731078904991948
In fold  2
Learning rate set to 0.074201
0:	learn: 0.4957667	test: 0.5084112	best: 0.5084112 (0)	total: 145ms	remaining: 2m 24s
200:	learn: 0.7375776	test: 0.7003155	best: 0.7014218 (192)	total: 23.8s	remaining: 1m 34s
400:	learn: 0.7800830	test: 0.6804734	best: 0.7024540 (318)	total: 50.3s	remaining: 1m 15s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.7024539877
bestIteration = 318

Shrink model to first 319 iterations.
Fold 2 score:  0.7024539877300613
In fold  3
Learning rate set to 0.074201
0:	learn: 0.5325550	test: 0.5366726	best: 0.5366726 (0)	total: 149ms	remaining: 2m 29s
20

In [37]:
ss = pd.read_csv(DIR + "ss.csv")
ss

,id,buy
0,39162,1
1,39163,1
2,39164,1
3,39165,1
4,39166,1
...,...,...
13179,52341,1
13180,52342,1
13181,52343,1
13182,52344,1


In [38]:
ss["buy"] = (preds > 0.5 ) * 1
ss["buy"].value_counts()

0    12858
1      326
Name: buy, dtype: int64

In [39]:
ss.to_csv("submission_cb_final.csv",index=False)